# Imports

In [1]:
!sudo /bin/bash -c "(source /venv/bin/activate; pip install gspread_pandas)"

In [2]:
!sudo /bin/bash -c "(source /venv/bin/activate; pip install google-api-python-client)"

In [2]:
import gspread_pandas

In [36]:
import logging
import helpers.hdbg as hdbg
import linkedin.google_api.google_file_api as google_file_api

In [37]:
_LOG = logging.getLogger(__name__)
hdbg.init_logger(use_exec_path=True)

# Load data

In [9]:
spreadsheet_name = "search_retired1.search_export.gsheet"
spread = gspread_pandas.Spread(spreadsheet_name)
df = spread.sheet_to_df(index=None)
print(df.shape)
df.head()

(14, 19)


,profileUrl,connectionDegree,error,currentJob,job,location,url,query,category,timestamp,fullName,firstName,lastName,profileImageUrl,summary,name,additionalInfo,pastJob,sharedConnections
0,,Out of Network,Profile out of your network.,Retired (Supervisory Environmental Engineer) a...,--,"Bel Air, MD",,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.384Z,,,,,,,,,
1,https://www.linkedin.com/in/yale-lewis-mba-049...,3rd,,,Retired in 2020,"Winchester, VA",https://www.linkedin.com/in/yale-lewis-mba-049...,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.384Z,"Yale Lewis, MBA",Yale,"Lewis, MBA",https://media.licdn.com/dms/image/C4D03AQGGXPi...,...Administration (MBA) Specialties: Financi...,"Yale Lewis, MBA",,,
2,https://www.linkedin.com/in/richard-saunders-9...,3rd,,,Retired from Federal Service.,"Silver Spring, MD",https://www.linkedin.com/in/richard-saunders-9...,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.384Z,Richard Saunders,Richard,Saunders,https://media.licdn.com/dms/image/D4E03AQHxFXh...,,Richard Saunders,,,
3,,Out of Network,Profile out of your network.,,"Retired, trying to figure out what's next","New Alexandria, VA",,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.384Z,,,,https://media.licdn.com/dms/image/C4E03AQEhKLx...,,,,,
4,https://www.linkedin.com/in/robertfarinella,3rd,,Retired at And Enjoying Life,US Army and Civilian Hospitality/Gaming/Horse ...,"Culpeper, VA",https://www.linkedin.com/in/robertfarinella,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.385Z,"Robert Farinella, MBA",Robert,"Farinella, MBA",https://media.licdn.com/dms/image/C4E03AQHPttO...,,"Robert Farinella, MBA",,,


# Filter data

In [10]:
# Drop errors.
prev_len = len(df)
df = df[df["error"] == ""].reset_index(drop=True)
df = df[[col for col in df.columns if col != "error"]]
print(
    f"Dropped {prev_len - len(df)} rows ({round((prev_len - len(df))*100/prev_len, 2)}%)"
)
df.head()

Dropped 4 rows (28.57%)


,profileUrl,connectionDegree,currentJob,job,location,url,query,category,timestamp,fullName,firstName,lastName,profileImageUrl,summary,name,additionalInfo,pastJob,sharedConnections
0,https://www.linkedin.com/in/yale-lewis-mba-049...,3rd,,Retired in 2020,"Winchester, VA",https://www.linkedin.com/in/yale-lewis-mba-049...,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.384Z,"Yale Lewis, MBA",Yale,"Lewis, MBA",https://media.licdn.com/dms/image/C4D03AQGGXPi...,...Administration (MBA) Specialties: Financi...,"Yale Lewis, MBA",,,
1,https://www.linkedin.com/in/richard-saunders-9...,3rd,,Retired from Federal Service.,"Silver Spring, MD",https://www.linkedin.com/in/richard-saunders-9...,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.384Z,Richard Saunders,Richard,Saunders,https://media.licdn.com/dms/image/D4E03AQHxFXh...,,Richard Saunders,,,
2,https://www.linkedin.com/in/robertfarinella,3rd,Retired at And Enjoying Life,US Army and Civilian Hospitality/Gaming/Horse ...,"Culpeper, VA",https://www.linkedin.com/in/robertfarinella,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.385Z,"Robert Farinella, MBA",Robert,"Farinella, MBA",https://media.licdn.com/dms/image/C4E03AQHPttO...,,"Robert Farinella, MBA",,,
3,https://www.linkedin.com/in/scottalawrence,3rd,,"Senior Manager, Software Engineering at Capita...","Silver Spring, MD",https://www.linkedin.com/in/scottalawrence,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.385Z,Scott L.,Scott,L.,https://media.licdn.com/dms/image/C4E03AQHI9E9...,,Scott L.,Volunteer Experience: Instructor at Adventist ...,,
4,https://www.linkedin.com/in/perrycoleman1,3rd,,"IT Specialist, MBA, MSCIS, PMP, CISSP, DML","Gwynn Oak, MD",https://www.linkedin.com/in/perrycoleman1,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.385Z,Perry M Coleman,Perry,M Coleman,https://media.licdn.com/dms/image/C4E03AQEJYFZ...,,Perry M Coleman,,Unmanned Aircraft System Operator at US Army -...,


In [6]:
# Filter by location.
# The location you want to keep.
location = "Maryland"
prev_len = len(df)
df = df[df["location"].apply(lambda x: location in x)].reset_index(drop=True)
print(
    f"Dropped {prev_len - len(df)} rows ({round((prev_len - len(df))*100/prev_len, 2)}%)"
)
df[:10]

Dropped 10 rows (100.0%)


,profileUrl,connectionDegree,currentJob,job,location,url,query,category,timestamp,fullName,firstName,lastName,profileImageUrl,summary,name,additionalInfo,pastJob,sharedConnections


In [11]:
for col in df.columns:
    print(col)
    print(df.iloc[0][col])
    print()

profileUrl
https://www.linkedin.com/in/yale-lewis-mba-049a0922

connectionDegree
3rd

currentJob


job
Retired in 2020

location
Winchester, VA

url
https://www.linkedin.com/in/yale-lewis-mba-049a0922

query
https://www.linkedin.com/search/results/people/?contactInterest=%5B%22proBono%22%5D&geoUrn=%5B%2290000097%22%5D&keywords=Retired%20mba&origin=FACETED_SEARCH&schoolFilter=%5B%223638%22%5D&sid=m7r

category
People

timestamp
2023-07-13T15:51:51.384Z

fullName
Yale Lewis, MBA

firstName
Yale

lastName
Lewis, MBA

profileImageUrl
https://media.licdn.com/dms/image/C4D03AQGGXPiY2Gy1Sw/profile-displayphoto-shrink_100_100/0/1516993465324?e=1694649600&v=beta&t=4nf-i7u5vxcVrXvNZRaGB-MuuzePNlhf_uTPoS0xiB8

summary
...Administration (MBA)  Specialties:  Financial Planning...

name
Yale Lewis, MBA

additionalInfo


pastJob


sharedConnections




In [9]:
# Make sure profileUrl in the first column.
cols = df.columns.tolist()
if cols[0] != 'profileUrl':
    cols.remove('profileUrl')  # Remove the "profileURL" from its current position.
    cols = ['profileUrl'] + cols  # Add "profileURL" at the first column.
    df = df[cols] 
df.head()

,profileUrl,query,timestamp,fullName,firstName,lastName,companyName,title,companyId,companyUrl,...,vmid,linkedInProfileUrl,isPremium,isOpenLink,titleDescription,pastExperienceCompanyName,pastExperienceCompanyUrl,pastExperienceCompanyTitle,pastExperienceDate,pastExperienceDuration
0,https://www.linkedin.com/sales/lead/ACwAAAE1Yq...,https://www.linkedin.com/sales/search/people?s...,2023-07-19T18:02:26.835Z,Neil Davis,Neil,Davis,Innovation Works,Business Mentor,11787752,https://www.linkedin.com/sales/company/11787752,...,ACwAAAE1YqQBZoy53iiEXT0Ol-GrNh_0U0dhvqM,https://www.linkedin.com/in/ACwAAAE1YqQBZoy53i...,FALSE,FALSE,Innovation Works aims to reduce Baltimore’s ne...,,,,,
1,https://www.linkedin.com/sales/lead/ACwAAACRHS...,https://www.linkedin.com/sales/search/people?s...,2023-07-19T18:02:26.835Z,Michael Hess,Michael,Hess,"ABS Advisory Services Group, Inc.",Chairman/CEO,,,...,ACwAAACRHSQB6TCtuIxEce2hAb9lV3NM_GzNPc8,https://www.linkedin.com/in/ACwAAACRHSQB6TCtuI...,FALSE,FALSE,"Manage mergers & acquisitions, corporate and r...",,,,,
2,https://www.linkedin.com/sales/lead/ACwAAAAQQk...,https://www.linkedin.com/sales/search/people?s...,2023-07-19T18:02:26.835Z,"Barrie Zucal, MS",Barrie,"Zucal, MS",Zucal Consulting,"Independent Mentor, Coach, Consultant",,,...,ACwAAAAQQkoBFSrQYiDk3QVKcRprtPvRSXe7ajk,https://www.linkedin.com/in/ACwAAAAQQkoBFSrQYi...,FALSE,FALSE,I work with talented women in global roles who...,,,,,
3,https://www.linkedin.com/sales/lead/ACwAAADeWj...,https://www.linkedin.com/sales/search/people?s...,2023-07-19T18:02:26.835Z,Leslie Traub,Leslie,Traub,Inclusion Allies Network,Co-Founder and Mentor,,,...,ACwAAADeWjEBRuyhZkIF7X7fpy6uBoxiXLQWqLY,https://www.linkedin.com/in/ACwAAADeWjEBRuyhZk...,FALSE,FALSE,A coalition of organizations and practitioner...,,,,,
4,https://www.linkedin.com/sales/lead/ACwAAAAY9Y...,https://www.linkedin.com/sales/search/people?s...,2023-07-19T18:02:26.836Z,Theresa Daytner,Theresa,Daytner,Conscious Venture Lab,Lead Mentor | Angel Investor | Executive Coach...,3356498,https://www.linkedin.com/sales/company/3356498,...,ACwAAAAY9YQBq8q_bnRNDw_10z49i6IR2gD7RKQ,https://www.linkedin.com/in/ACwAAAAY9YQBq8q_bn...,TRUE,FALSE,As a Lead Mentor for the Conscious Venture Lab...,,,,,


In [10]:
print(df.shape)

(82, 31)


# Save filtered data to gsheet

## Create a empty Google sheet

In [18]:
gapi = google_file_api.GoogleFileApi()

INFO  file_cache is only supported with oauth2client<4.0.0


In [24]:
name = 'SN_Search5_Yiyun'
# gdrive_folder : dict, the id and the name of the Google Drive folder.
gdrive_folder  = gapi.get_folder_id_by_name(name)

INFO  Found folder: {'id': '1XWNGDnJrVICHAe-6V2cnoSklZpk0APc_', 'name': 'SN_Search5_Yiyun'}


In [25]:
# if you want to use another folder id, please change the folder id manually.
# gdrive_folder  = {'id': '1XWNGDnJrVICHAe-6V2cnoSklZpk0APc_', 'name': 'SN_Search5_Yiyun'} 

In [27]:
"""
Create a new Google file (sheet or doc).

:param gfile_type: str, the type of the Google file ('sheet' or 'doc').
:param gfile_name: str, the name of the new Google file.
:param folder_id: str, the id of the Google Drive folder.
:param user: str, the email address of the user to share the Google file (Optional).
:return: None
"""
gfile_type = 'sheet'
gsheet_name = 'sn_search5.search_export.filtered.gsheet'
user = ''

In [29]:
gapi.create_empty_google_file(
    gfile_type = gfile_type,
    gfile_name = gsheet_name,
    gdrive_folder = gdrive_folder,
    user = user
)

INFO  file_cache is only supported with oauth2client<4.0.0
INFO  Created a new Google sheet 'sn_search5.search_export.filtered.gsheet'.
INFO  Move the new Google sheet 'sn_search5.search_export.filtered.gsheet' to the dir 'SN_Search5_Yiyun'
INFO  Finished creating the new Google sheet 'sn_search5.search_export.filtered.gsheet'.


## Save filtered data

In [30]:
# A Google sheet with this name should already exist on the drive.
spread2 = gspread_pandas.Spread(
    gsheet_name,
    sheet="Sheet1",
    create_sheet=True,
)
spread2.df_to_sheet(df, index=False)